In [ ]:
from transformers import BertTokenizer, BertModel, BertConfig, BertForSequenceClassification
from transformers import BartTokenizer, BartModel, BartConfig, BartForSequenceClassification
from transformers import RobertaTokenizer, RobertaModel, RobertaConfig, RobertaForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, precision_score, recall_score, precision_recall_curve
from sklearn.metrics import confusion_matrix, classification_report
from torch.utils.data import TensorDataset, DataLoader, Dataset
import pandas as pd
import numpy as np
import torch.nn as nn
import torch
import random

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


**Model loading**

In [ ]:
MODEL_TYPE    = 'bert-base-uncased'
tokenizer_b     = BertTokenizer.from_pretrained(MODEL_TYPE)
model_b = BertForSequenceClassification.from_pretrained(MODEL_TYPE, num_labels = 3)
model_b.load_state_dict(torch.load('/content/drive/My Drive/Gender_Bias_NLI/Models/MNLI_BERT.bin', map_location=torch.device('cpu')))
model_b = model_ba.to(device)

In [ ]:

MODEL_TYPE    = 'facebook/bart-base'
tokenizer_ba     = BartTokenizer.from_pretrained(MODEL_TYPE)
model_ba = BartForSequenceClassification.from_pretrained(MODEL_TYPE, num_labels = 3)
model_ba.load_state_dict(torch.load('/content/drive/My Drive/Gender Bias NLI Final/Models/MNLI_BART.bin', map_location=torch.device('cpu')))
model_ba = model_ba.to(device)

In [ ]:
MODEL_TYPE    = 'roberta-base'
tokenizer_r     = RobertaTokenizer.from_pretrained(MODEL_TYPE)
model_r = RobertaForSequenceClassification.from_pretrained(MODEL_TYPE, num_labels = 3)
model_r.load_state_dict(torch.load('/content/drive/My Drive/Gender_Bias_NLI/Models/MNLI_Roberta.bin', map_location=torch.device('cpu')))
model_r = model_r.to(device)

**Evaluation set loading**

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Gender Bias NLI Final/Datasets/MNLI_Evaluation_Set.csv')

In [ ]:
def get_predictions(s1, s2, model, tokenizer):
  encoded_seq= tokenizer.encode_plus(
    s1,
    s2,
    max_length=180,
    add_special_tokens=True,
    return_token_type_ids=True,
    pad_to_max_length=True,
    return_attention_mask=True,
    return_tensors='pt',
    truncation=True
  )
  input_ids = encoded_seq['input_ids'].to(device)
  attention_mask = encoded_seq['attention_mask'].to(device)
  # token_type_ids= encoded_seq['token_type_ids'].to(device)


  output = model(
        input_ids,
        attention_mask,
        # token_type_ids
        
      )

  logit = output[0]
  entail_contradiction_logits = logit[:,[0,2]]
  probs = entail_contradiction_logits.softmax(dim=1)

  entail_prob = probs[:,0].item()*100
  cont_prob = probs[:,1].item()*100

  entail_contradiction_logits = entail_contradiction_logits.detach().cpu().numpy()
  prediction = np.argmax(entail_contradiction_logits, axis=1)


  return prediction, entail_prob, cont_prob

In [ ]:
predictions_1 = []
predictions_2 = []

e_1 = []
e_2 = []

c_1 = []
c_2 = []

for i,r in df.iterrows(): 
  prediction, e, c = get_predictions(r['sentence1'],r['hypothesis_f'], model_r, tokenizer_r)
  predictions_1.append(prediction)
  e_1.append(e)
  c_1.append(c)

  prediction, e, c = get_predictions(r['sentence1'],r['hypothesis_m'], model_r, tokenizer_r)
  # print(e,n,c)
  predictions_2.append(prediction)
  e_2.append(e)
  c_2.append(c)
  if np.sum(e_1)>np.sum(e_2):
    print(np.sum(e_1)-np.sum(e_2))

In [ ]:
df['f_r_pred'] = predictions_1
df['m_r_pred'] = predictions_2

df['f_r_0'] = e_1
df['m_r_0'] = e_2

df['f_r_1'] = c_1
df['m_r_1'] = c_2

In [ ]:
predictions_1 = []
predictions_2 = []

e_1 = []
e_2 = []

c_1 = []
c_2 = []

for i,r in df.iterrows(): 
  prediction, e, c = get_predictions(r['sentence1'],r['hypothesis_f'], model_a, tokenizer_a)
  predictions_1.append(prediction)
  e_1.append(e)
  c_1.append(c)

  prediction, e, c = get_predictions(r['sentence1'],r['hypothesis_m'], model_a, tokenizer_a)
  # print(e,n,c)
  predictions_2.append(prediction)
  e_2.append(e)
  c_2.append(c)
  if np.sum(e_1)>np.sum(e_2):
    print(np.sum(e_1)-np.sum(e_2))

In [ ]:
df['f_a_pred'] = predictions_1
df['m_a_pred'] = predictions_2

df['f_a_0'] = e_1
df['m_a_0'] = e_2

df['f_a_1'] = c_1
df['m_a_1'] = c_2

In [ ]:
predictions_1 = []
predictions_2 = []

e_1 = []
e_2 = []

c_1 = []
c_2 = []

for i,r in df.iterrows(): 
  prediction, e, c = get_predictions(r['sentence1'],r['hypothesis_f'], model_ba, tokenizer_ba)
  predictions_1.append(prediction)
  e_1.append(e)
  c_1.append(c)

  prediction, e, c = get_predictions(r['sentence1'],r['hypothesis_m'], model_ba, tokenizer_ba)
  # print(e,n,c)
  predictions_2.append(prediction)
  e_2.append(e)
  c_2.append(c)
  if np.sum(e_1)>np.sum(e_2):
    print(np.sum(e_1)-np.sum(e_2))

In [ ]:
df['f_ba_pred'] = predictions_1
df['m_ba_pred'] = predictions_2

df['f_ba_0'] = e_1
df['m_ba_0'] = e_2

df['f_ba_1'] = c_1
df['m_ba_1'] = c_2

In [ ]:
df.to_csv('/content/drive/My Drive/Gender Bias NLI Final/Results/MNLI_Predictions_Final.csv')